<a href="https://colab.research.google.com/github/rafaelchiaradia/Projeto-1-Spam-Prediction/blob/main/MestradoRafaelv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### Mastering: Rafael Chiaradia Almeida
#### Advisor: Glauco de Figueiredo Carneiro
#### Co-Advisor: Edward David Moreno

In [1]:
package_list = "https://catalogodatos.gub.uy/api/3/action/package_list" 
##List with all the dataset ID´s in dados.gov.br portal

import requests
import json
import re
response = requests.get(package_list, verify=False)
print(response)
import pandas as pd
import numpy as np
data=response.json() ## Save that list in a variable
data = data["result"] ## Save only the dataset ID´s in a variable

type (data) ## Now we have a list with all the dataset ID´s in the portal
len(data) ## number of datasets in the the portal
##data[0] ## now, all we have to do is to index the dataset we want to access
##data[len(data)-1] ## from 0 to the last one

M3 = [] ## variable that will save the metric M3
M5 = [] ## variable that will save the metric M5
M6 = [] ## variable that will save the metric M6
M7 = [] ## variable that will save the metric M7
M25 = [] ## variable that will save the metric M25

## the next step now is to access each dataset id using package_show

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'catalogodatos.gub.uy'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


In [2]:
len (data)

2446

In [ ]:
from pandas.core.api import notnull
from pandas.core.base import value_counts 
for i in range(0, (len(data)-1)):
## for i in range(0, 5):
##loop for that will run into all datasets

  dataset = []
  package_show = package_list.replace("package_list","package_show?id=")
  package_show = package_show + data[i]
  response2 = requests.get(package_show, verify=False)
  if response2.status_code == 200:
    dataset=response2.json()

    packages = dataset["result"]
    
    ## let´s create separate dataframes for the metadata keys hierarchy
    ## after the 3 keys that have subkeys were saved in other variables
    ## we can remove then from packages dataset
    
    if 'resources' in packages:
      resources = packages["resources"]
      del packages["resources"]
    else:
      resources = {}
    if 'tags' in packages:
      tags = packages["tags"]
      del packages["tags"]
    else:
      tags = {}
    if 'organization' in packages:
      organization = packages["organization"]
      del packages["organization"]
    else:
      organization = {}


    ## let´s transform the variables into dataframe for further analysis
    dfpackages = pd.DataFrame(list(packages.items()))
    dfpackages = dfpackages.T
    dfpackages.columns = dfpackages.iloc[0]
    dfpackages = dfpackages[1:]
    dfresources = pd.DataFrame(resources)
    dftags = pd.DataFrame(tags)
    if (organization is not None) and (organization != {}):
      dforganization = pd.DataFrame(list(organization.items()))
      dforganization = dforganization.T
      dforganization.columns = dforganization.iloc[0]
      dforganization = dforganization[1:]
    else:
      dforganization = pd.DataFrame()
    
    ## We now have all metadata keys saved in 4 dataframes: dfpackages, dfresources, dftags e dforganization
    ## Now we can start calculating the metrics

    #######################################################################################################
    ################################## TIMELINESS #########################################################
    #######################################################################################################

    ################# M3 - Percentage of time metadata cells that has information #########################

    ## In the CKAN metadata scheme there are two keys inside the RESOURCE metadata set that are related 
    ## with time. One called CREATED, which saves the date and time the dataset was created, and another 
    ## called LAST_MODIFIED, which saves the data and time the dataset was last modified. So, in our 
    ## script we check those metadata keys, if both are filled in we grade it one, if only one is filled in 
    ## we apply a score of 0.5, and if none of them has information we grade it zero. If the dataset has 
    ## more than one resource, an average is taken and that will be the M3 grade of the dataset. In the 
    ## end, the M3 grade of the portal will be the average of the M3 grade of each dataset.

    grade=0
    if len(resources) != 0:
      for chave, valor in dfpackages['num_resources'].iteritems():
        if (valor != 0):  
          if ('created' in dfresources):
            for key, value in dfresources['created'].iteritems():
              if (value is not None):
                grade = grade + 0.5
          if ('last_modified' in dfresources):
            for key, value in dfresources['last_modified'].iteritems():
              if (value is not None):
                grade = grade + 0.5
 
        grade = grade / len (dfresources['created'].index)

      M3.append (grade)
    

    #######################################################################################################
    #################################### FORMAT ###########################################################
    #######################################################################################################

    ################# M5 - Dataset files are machine-readable and downloadable ############################

    ## In the CKAN metadata scheme there is a key called FORMAT, inside the RESOURCE metadata set, 
    ## which saves the dataset file format information. In our script we checked that metadata key and 
    ## graded metric M5 using Tim Berners-Lee 5-Star Open Data scheme as a basis. 
    ## So, if the FORMAT field is blank, we grade 0 to M5. If it is filled with "lod", we grade 1. 
    ## For "rdf" or "xml", we grade 0.8. When it has "csv" or "json", we grade 0.6. 
    ## For "xls" the grade is 0.4 and "pdf" or any other format we grade 0.2. 
    ## As a package can have more than one resource, we had to create a loop to go through the resources 
    ## and if there is more than one, an average is taken and saved into M5.

  
    grade=0
    if len(resources) != 0:
      for chave, valor in dfpackages['num_resources'].iteritems():
        if (valor != 0):  
          for key, value in dfresources['format'].iteritems():
            if (value == "LOD"):
              grade = grade + 1
            elif (value == "RDF" or value == "XML"):
              grade = grade + 0.8
            elif (value == "CSV" or value == "JSON"):
              grade = grade + 0.6
            elif (value == "XLS"):
              grade = grade + 0.4
            elif (value == "null"):
              grade = grade + 0
            else:
              grade = grade + 0.2
 
          grade = grade / len (dfresources['format'].index)
      
      M5.append (grade)

     ################# M6 - Dataset license format conform to the open definition #########################

     ## What we did was save those license´s name and their variations in a variable 
     ## and then check if the metadata LICENSE_TITLE or LICENSE_ID content of the dataset
     ## matches with one of the conformant licenses. If YES it gets a grade 1, 
     ## and if NO it gets 0. In the end an average is taken and that is the final M6 grade
     ## of the OGD portal.

    licenses_list = [
        'Creative Commons CCZero','CC0-1.0','Open Data Commons Public Domain Dedication and Licence', 
        'PDDL-1.0','Creative Commons Attribution 4.0','CC-BY-4.0','Open Data Commons Attribution License',
        'ODC-By-1.0','Creative Commons Attribution Share-Alike 4.0','CC-BY-SA-4.0',
        'Open Data Commons Open Database License','ODbL-1.0','Against DRM',
        'Creative Commons Attribution 1.0','CC-BY-1.0','Creative Commons Attribution 2.0','CC-BY-2.0',
        'Creative Commons Attribution 3.0','CC-BY-3.0','Creative Commons Attribution Share-Alike 1.0',
        'CC-BY-SA-1.0','Creative Commons Attribution Share-Alike 2.0','CC-BY-SA-2.0',
        'Creative Commons Attribution Share-Alike 3.0','CC-BY-SA-3.0',
        'Data licence Germany – attribution – version 2.0','Data licence Germany – Zero – version 2.0',
        'Design Science License','EFF Open Audio License','Free Art License 1.2','Free Art License 1.3',
        'LAL-1.2','LAL-1.3','GNU Free Documentation License 1.1','GFDL-1.1','GNU Free Documentation License 1.2',
        'GFDL-1.2','GNU Free Documentation License 1.3','GFDL-1.3','MirOS License','MirOS',
        'Open Government Licence Canada 2.0','OGL-Canada-2.0','Open Government Data License Taiwan 1.0',
        'OGDL-Taiwan-1.0','Open Government Licence United Kingdom 2.0','Open Government Licence United Kingdom 3.0',
        'OGL-UK-2.0','OGL-UK-3.0','Open Use of Data Agreement 1.0','O-UDA-1.0','Talis Community License'
                     ] 
    grade=0
    for n in licenses_list:
      if ('license_id' in dfpackages):
        for chave, valor in dfpackages['license_id'].iteritems():
          if (valor is not None):
            if (n in valor and grade != 1):  
              grade = grade + 1
            else:
              if ('license_title' in dfpackages):
                for key, value in dfpackages['license_title'].iteritems():
                  if (value is not None):
                    if (n in value and grade != 1):  
                      grade = grade + 1
          else:
            if ('license_title' in dfpackages):
              for key, value in dfpackages['license_title'].iteritems():
                if (value is not None):
                  if (n in value and grade != 1):  
                    grade = grade + 1       
        
    M6.append (grade)

    #############################################################################################################
    #################################### COMPLETENESS ###########################################################
    #############################################################################################################

    ################# M7 - Percentage of metadata cells that has information ####################################

    ## To grade this metric we had to write a script that checks each metadata key and count the amount of 
    ## NULL/NA values. Then, we calculate how much those values represents over the total metadata fields. 
    ## We discovered that there were many metadata fields that were filled with characters such as '?', '-', 
    ## or a blank space; and, before using the ISNULL function to count the amount of NULL and NaN values, 
    ## we had to convert those characters into NaN values using the REPLACE instruction, otherwise they would 
    ## be count as valid information.

    grade=0
    dfpackages = dfpackages.replace({"?":np.nan, "":np.nan,"-":np.nan })
    dfresources = dfresources.replace({"?":np.nan, "":np.nan,"-":np.nan })
    dftags = dftags.replace({"?":np.nan, "":np.nan,"-":np.nan })
    dforganization = dforganization.replace({"?":np.nan, "":np.nan,"-":np.nan })
    totalNULLvalues = dfpackages.isnull().sum().sum() + dfresources.isnull().sum().sum() 
    + dftags.isnull().sum().sum() + dforganization.isnull().sum().sum() 
    totalmetadatakeys = dfpackages.shape[0]*dfpackages.shape[1] + dfresources.shape[0]*dfresources.shape[1]
    + dftags.shape[0]*dftags.shape[1] + dforganization.shape[0]*dforganization.shape[1]
    grade = 1 - (totalNULLvalues/totalmetadatakeys)

    M7.append (grade)



    #######################################################################################################
    ################################### DUPLICITY #########################################################
    #######################################################################################################

    ####################### M25 - Percentage of duplicated metadata cells #################################

    ## For this metric we checked the four sets of metadata (PACKAGES, RESOURCES, TAGS and ORGANIZATION) 
    ## looking for duplicated cells. The final value of M25 is the quantity of duplicities divided by the 
    ## total metadata cells.

    from collections import Counter

    grade=0
    duplicities=0
    totalmetadatacells=0

    totalmetadatacells = dfpackages.shape[1] + dfresources.shape[1] + dftags.shape[1] + dforganization.shape[1]
   
    
    for key, value in Counter(list(dfpackages.columns)).items():
      if (value != 1):
        duplicities = duplicities + 1 
    for key, value in Counter(list(dfresources.columns)).items():  
      if (value != 1):
        duplicities = duplicities + 1
    for key, value in Counter(list(dftags.columns)).items():  
      if (value != 1):
        duplicities = duplicities + 1
    for key, value in Counter(list(dforganization.columns)).items():  
      if (value != 1):
        duplicities = duplicities + 1

    grade = 1 - duplicities/totalmetadatacells

    M25.append (grade)









## final do for que percorre todos os datasets


/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'catalogodatos.gub.uy'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'catalogodatos.gub.uy'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'catalogodatos.gub.uy'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/urllib3/connecti

In [32]:
totalmetadatacells




59

In [ ]:
dfresources.isnull().sum()


mimetype              2
cache_url             2
hash                  2
description           1
name                  0
format                0
url                   0
datastore_active      0
cache_last_updated    2
package_id            0
created               0
state                 0
mimetype_inner        2
last_modified         0
position              0
revision_id           0
url_type              0
id                    0
resource_type         2
size                  2
dtype: int64

In [ ]:
M25




In [ ]:
from collections import Counter
Counter (M6)

Counter({0: 28})

In [ ]:
len (M25)

In [ ]:
## In the end the final grade of the portal is the average of datasets grade

from statistics import mean
mean (M25)


In [ ]:
with open('datosgubuyM25.json', 'w') as f:
  json.dump(M25, f)